# Summary

Adapter model training and evaluation modeled after Don't Stop Pretraining paper

# Setup

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
!pip install datasets
!pip install huggingface_hub
!pip install scikit-learn
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64

In [2]:
!pip install -qq adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.0/256.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.7 MB/s eta 0:00:00


In [3]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import accuracy_score,  f1_score

from datasets import load_dataset

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Create function for metrics for evaluation

In [4]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

Load the tokenizer - Roberta Base

In [5]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")#, truncation=True, padding=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

512

In [6]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained("roberta-base")

# Training

## DAPT

Continue pretraining ROBERTA on a large corpus of unlabeled
domain-specific text (Reviews dataset).

In [7]:
from datasets import load_dataset, DatasetDict

# split = 'train[0:20%]' # reduce the working size to speed up iteration

# dataset_train = load_dataset("BigTMiami/amazon_25M_reviews_condensed", split='train[0:20%]')
# dataset_valid = load_dataset("BigTMiami/amazon_25M_reviews_condensed", split='dev[0:20%]')
# dataset_test = load_dataset("BigTMiami/amazon_25M_reviews_condensed", split='test[0:20%]')

# domain_dataset = load_dataset("BigTMiami/amazon_split_25M_reviews_20_percent_condensed")

domain_dataset = load_dataset("BigTMiami/amazon_split_25M_reviews_20_percent_condensed")
# # 90% train, 10% test + validation
# train_testvalid = full_dataset.train_test_split(test_size=0.1)

# # Split the 10% test + valid in half test, half valid
# test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# # gather everyone if you want to have a single DatasetDict
# domain_dataset = DatasetDict({'train': train_testvalid['train'], 'test': test_valid['test'], 'validation': test_valid['train']})

Generating train split:   0%|          | 0/862683 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8360 [00:00<?, ? examples/s]

In [ ]:
# Process domain dataset to be batch size
domain_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 862683
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8360
    })
})

In [8]:

tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)#, truncation=True, padding=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
# def encode_batch(batch):
#   """Encodes a batch of input data using the model tokenizer."""
#   return tokenizer(batch, max_length=32, truncation=True, padding="max_length")

# # Encode the input data
# dataset = dataset.map(encode_batch, batched=True)

Map:   0%|          | 0/862683 [00:00<?, ? examples/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

## Basic Architecture

In [ ]:
from adapters import AutoAdapterModel
# Model
dapt_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter to model - use seq bn architecture
dapt_model.add_adapter("review_adapter_pfeiffer", config="seq_bn")

# # Add a matching classification head
# dapt_model.add_classification_head(
#     "review_adapter",
#     num_labels=2,
#     id2label={ 0: "👎", 1: "👍"}
#   )

# Activate the adapter
dapt_model.train_adapter("review_adapter_pfeiffer")
dapt_model.set_active_adapters("review_adapter_pfeiffer") # this is unnecessary if call train

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("roberta-base")#, padding="max_length")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

# lm_datasets = domain_dataset.map(
#     group_texts,
#     batched=True,
#     batch_size=32,
#     num_proc=4,
# )

# #tokenize dataset with padding and truncation
# dataset_tokenized = domain_dataset.map(tokenizer, batched=True)

# #Instantiate Pytorch DataLoader
# dl = DataLoader(dataset_tokenized, shuffle=True, collate_fn=data_collator, batch_size=32)


In [18]:
# Training
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_dapt_reviews_small",
    learning_rate=0.0005,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=22, # 16*128*6
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="steps",
    eval_steps=500, # Evaluates every 50 steps
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progres
)


# def compute_accuracy(p: EvalPrediction):
#   preds = np.argmax(p.predictions, axis=1)
#   return {"acc": (preds == p.label_ids).mean()}

dapt_trainer = AdapterTrainer(
    model=dapt_model,
    args=training_args,
    # tokenizer=tokenizer,
    train_dataset=domain_dataset["train"],
    eval_dataset=domain_dataset["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 8.38 MiB is free. Process 4114 has 15.76 GiB memory in use. Of the allocated memory 15.36 GiB is allocated by PyTorch, and 41.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
eval_results = dapt_trainer.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
# print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Non Trained Eval Loss: 1.90
{'eval_loss': 1.8973259925842285, 'eval_runtime': 90.9474, 'eval_samples_per_second': 91.921, 'eval_steps_per_second': 2.881}


In [ ]:
# TRAINING
results = dapt_trainer.train()
print(results)

Step,Training Loss,Validation Loss
500,1.700000,1.598184
1000,1.687300,1.585015
1500,1.673500,1.577673
2000,1.668600,1.572553


TrainOutput(global_step=2450, training_loss=1.690375188710738, metrics={'train_runtime': 19352.2036, 'train_samples_per_second': 44.578, 'train_steps_per_second': 0.127, 'total_flos': 2.309746532696064e+17, 'train_loss': 1.690375188710738, 'epoch': 1.0})


In [ ]:
dapt_trainer.evaluate()

{'eval_loss': 1.5694469213485718,
 'eval_runtime': 90.8478,
 'eval_samples_per_second': 92.022,
 'eval_steps_per_second': 2.884,
 'epoch': 1.0}

In [ ]:
dapt_trainer.push_to_hub("RobertaAdapter_reviews_DAPT_v1")

## Adapter DAPT - Different Architectures


Arch 1 - pfeiffer https://arxiv.org/abs/2005.00247

https://github.com/adapter-hub/efficient-task-transfer

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from adapters import AutoAdapterModel
# Model
dapt_model_pfeiffer = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter to model - use seq bn architecture
dapt_model_pfeiffer.add_adapter("review_adapter", config="pfeiffer")

# # Add a matching classification head
# dapt_model.add_classification_head(
#     "review_adapter",
#     num_labels=2,
#     id2label={ 0: "👎", 1: "👍"}
#   )

# Activate the adapter
dapt_model_pfeiffer.train_adapter("review_adapter")
dapt_model_pfeiffer.set_active_adapters("review_adapter") # this is unnecessary if call train

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Training
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_dapt_reviews_pfeiffer",
    learning_rate=0.0005,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=22, # 16*128*6
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="steps",
    eval_steps=500, # Evaluates every 50 steps
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progres
)


# def compute_accuracy(p: EvalPrediction):
#   preds = np.argmax(p.predictions, axis=1)
#   return {"acc": (preds == p.label_ids).mean()}

dapt_trainer_pfeiffer = AdapterTrainer(
    model=dapt_model_pfeiffer,
    args=training_args,
    # tokenizer=tokenizer,
    train_dataset=domain_dataset["train"],
    eval_dataset=domain_dataset["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [14]:
eval_results = dapt_trainer_pfeiffer.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
# print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Non Trained Eval Loss: 1.89
{'eval_loss': 1.886637568473816, 'eval_runtime': 117.3966, 'eval_samples_per_second': 71.212, 'eval_steps_per_second': 2.232}


In [ ]:
# TRAINING
results = dapt_trainer_pfeiffer.train()
print(results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
dapt_trainer_pfeiffer.evaluate()

Arch 2 - houlsby https://arxiv.org/abs/1902.00751

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from adapters import AutoAdapterModel
# Model
dapt_model_houlsby = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter to model - use seq bn architecture
dapt_model_houlsby.add_adapter("review_adapter", config="houlsby")

# # Add a matching classification head
# dapt_model.add_classification_head(
#     "review_adapter",
#     num_labels=2,
#     id2label={ 0: "👎", 1: "👍"}
#   )

# Activate the adapter
dapt_model_houlsby.train_adapter("review_adapter")
dapt_model_houlsby.set_active_adapters("review_adapter") # this is unnecessary if call train

In [ ]:
# Training
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    output_dir="./adapter_dapt_reviews_houlsby",
    learning_rate=0.0005,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=22, # 16*128*6
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="steps",
    eval_steps=500, # Evaluates every 50 steps
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progres
)


# def compute_accuracy(p: EvalPrediction):
#   preds = np.argmax(p.predictions, axis=1)
#   return {"acc": (preds == p.label_ids).mean()}

dapt_trainer_houlsby = AdapterTrainer(
    model=dapt_model_houlsby,
    args=training_args,
    # tokenizer=tokenizer,
    train_dataset=domain_dataset["train"],
    eval_dataset=domain_dataset["validation"],
    data_collator=data_collator,
)

In [ ]:
eval_results = dapt_trainer_houlsby.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
# print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
# TRAINING
results = dapt_trainer_houlsby.train()
print(results)

In [ ]:
dapt_trainer_houlsby.evaluate()

Arch 3 - try pfeiffer+inv and houlsby+inv

# TAPT

In [ ]:
task_dataset = load_dataset("BigTMiami/amazon_helpfulness")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

In [ ]:
task_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 115251
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [ ]:
task_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=classification_config,
)

# Add a new adapter to model
task_model.add_adapter("task_review_helpfulness_adapter", config="seq_bn")

# Add a matching classification head
task_model.add_classification_head(
    "reviews_helpfulness_adapter",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
task_model.train_adapter("task_review_helpfulness_adapter")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'heads.default.3.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Use Adapter Training Class
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=0.0001,
    adam_epsilon=1e-6,
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=128, # notes on pretraining
    #logging_steps=200,
    adam_beta1 = 0.9,
    adam_beta2 = 0.98,
    weight_decay = 0.01,
    warmup_ratio = 0.06,
    # masking_probability=0.15,
    lr_scheduler_type="linear",
    output_dir="./adapter_tapt_helpfulness",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

task_trainer = AdapterTrainer(
    model=task_model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=task_dataset["train"],
    eval_dataset=task_dataset["dev"],
    compute_metrics=compute_accuracy,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
# TRAINING
task_trainer.train()

ValueError: Expected input batch_size (16) to match target batch_size (8192).

In [ ]:
# Evaluate
task_trainer.evaluate()

In [ ]:
task_trainer.push_to_hub("RobertaAdapter_helpfulness_TAPT")

In [ ]:
# Save adapter to hugging face
task_trainer.save_model("RobertaAdapter_helpfulnessModel")

# Evaluation

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=task_model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

In [ ]:
# Save trained adapter
task_model.save_adapter("./test_adapter1", "rotten_tomatoes")

!ls -lh final_adapter